In [572]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [574]:
dataset = pd.read_csv("amazon.csv")

In [578]:
user_review = dataset[["about_product", "product_id"]]

In [580]:
user_review = user_review.rename(columns={'about_product': 'review_text', 'product_id': 'product_id'})

In [ ]:
#user_review ile user_rating'i birleştiricez ileride

In [582]:
user_rating = dataset[["rating","product_id"]]

In [586]:
user_rating = user_rating.rename(columns={"rating" : "rating","product_id" : "product_id"})

In [ ]:
#Boşlukları 0 ile doldurma
user_rating['rating'].fillna(0, inplace=True)

In [ ]:
#VERİ SETİNİ OLUŞTURDUK

In [ ]:
#VERİ ÖN İŞLEME

In [ ]:
def preprocess_dataset(data):
    #url
    data = re.sub(r"http\S+|www\S+|https\S+", "", data)
    #hashtag 
    data = re.sub(r"\#\w+", "", data)
    #sayılar
    data = re.sub(r"\d+", "", data)
    #noktalama
    data = re.sub(r"[^\w\s]", "", data)
    #fazla boşluklar
    data = re.sub(r"\s+", " ", data).strip()
    return data

In [ ]:
#Ön işlemeyi başlattık...
user_review["review_text"] = [preprocess_dataset(user_review) for user_review in user_review['review_text']]

In [ ]:
#TOKENİZASYON

In [ ]:
def tokenization(data):
    tokenized_text = data.split()
    return tokenized_text

In [ ]:
# Her bir satıra preprocess_dataset fonksiyonunu uygula
user_review["review_text"] = user_review["review_text"].apply(tokenization)


In [ ]:
#BÜTÜN HARFLERİ KÜÇÜK HARF YAPMA

In [ ]:
user_review["review_text"] = user_review["review_text"].apply(lambda lists: [word.lower() for word in lists])

In [ ]:
#BOŞLUKLARI SİLME

In [ ]:
user_review["review_text"] = user_review["review_text"].apply(lambda lists: [word.strip(' ') for word in lists])


In [ ]:
#STOPWORDS

In [ ]:
nltk.download('stopwords')

In [ ]:
stopwords = set(stopwords.words('english'))

In [ ]:
user_review["review_text"] = user_review["review_text"].apply(lambda lists: [word for word in lists if not word in stopwords])


In [588]:
#SCORE İSİMLİ DEĞİŞKEN EKLENDİ

In [598]:
user_review["score"] = ""

In [611]:
user_review["review_text"] = user_review["review_text"].apply(lambda txt: ' '.join(txt))

In [612]:
#DUYGU ANALİZİ

In [613]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\samet\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [615]:
vader_analyzer = SentimentIntensityAnalyzer()

In [617]:
user_review["score"] = user_review["review_text"].apply(lambda lists: vader_analyzer.polarity_scores(lists))

In [618]:
user_review_score = pd.json_normalize(user_review["score"])

In [619]:
user_review = pd.concat([user_review.drop(columns=["score"]), user_review_score], axis=1)

In [620]:
#KİŞİLERİN VERDİĞİ DEĞERLENDİRME PUANI İLE YAZDIKLARININ DEĞERLENDİRMELERİN POLARİTESİ AYNI DATAFRAME ALDIK

In [621]:
df = pd.merge(user_review,user_rating, on = "product_id")

In [622]:
df = df.drop_duplicates()